In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
pd.set_option("display.max_rows", None)

In [7]:
class CrawlCommodityTaifex:
    
    @classmethod
    def stock_relate(cls):
        r = requests.post('https://www.taifex.com.tw/cht/2/stockLists')
        r.encoding = 'utf8'
        lines = r.text.replace('\r', '').split('\n')
        content = "\n".join(lines)
        df = pd.read_html(StringIO(content))
        df=pd.DataFrame(df[0])
        df=df.astype(str)
        df['證券代號']=df['證券代號'].apply(lambda s:s[:-2])
        df=df.replace('●',1).replace('◎',1)
        df['標準型證券股數']=df['標準型證券股數'].apply(lambda s:pd.to_numeric(s, errors='coerce'))
        df=df.drop(columns=['標的證券'])
        df['標的證券簡稱']=df['標的證券簡稱'].apply(lambda s:s+'期')
        df=df.rename(columns={k:v for k,v in zip(df.columns,['stock_id','spot_id','stock_name','check_fc',
                                                             'check_opt','check_sii','check_otc','check_etf',
                                                             'spot_unit'])})        
        return df
    
    @staticmethod
    def modify_df(df):
        if '中文簡稱.1' in df.columns:
            df['中文簡稱']=[a if a==b else a+b  for a,b in zip(df['中文簡稱'].values,df['中文簡稱.1'].values)]
            df=df.drop(columns=['中文簡稱.1'])
        return df
    
    @classmethod
    def normal(cls):
        r = requests.post('https://www.taifex.com.tw/cht/4/contractName')
        r.encoding = 'utf8'
        lines = r.text.replace('\r', '').split('\n')
        content = "\n".join(lines)
        df = pd.read_html(StringIO(content))
        df_all=pd.concat([cls.modify_df(pd.DataFrame(df[i])) for i in range(len(df) )])
        df_all=df_all.dropna(thresh=8,axis=1).dropna(how='any',axis=0)
        df_all=df_all.rename(columns={k:v for k,v in zip(df_all.columns,['stock_name','stock_id'])})         
        return df_all
    
    @classmethod    
    def crawl_main(cls):
        try:
            df = pd.concat([cls.stock_relate(), cls.normal()])
        except ValueError:
            return None
        return df    
    
CrawlCommodityTaifex.crawl_main()

,stock_id,spot_id,stock_name,check_fc,check_opt,check_sii,check_otc,check_etf,spot_unit
0,CA,1303,南亞期,1,1,1,nan,nan,2000.0
1,CB,2002,中鋼期,1,1,1,nan,nan,2000.0
2,CC,2303,聯電期,1,1,1,nan,nan,2000.0
3,CD,2330,台積電期,1,1,1,nan,nan,2000.0
4,CE,2881,富邦金期,1,1,1,nan,nan,2000.0
5,CF,1301,台塑期,1,1,1,nan,nan,2000.0
6,CG,2324,仁寶期,1,1,1,nan,nan,2000.0
7,CH,2409,友達期,1,1,1,nan,nan,2000.0
8,CJ,2880,華南金期,1,1,1,nan,nan,2000.0
9,CK,2882,國泰金期,1,1,1,nan,nan,2000.0
